In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Download model
!curl "https://civitai-delivery-worker-prod.5ac0637cfd0766c97916cefa3764fbdf.r2.cloudflarestorage.com/model/31176/waiNsfwIllustrious15.tvs0.safetensors?X-Amz-Expires=86400&response-content-disposition=attachment%3B%20filename%3D%22waiNSFWIllustrious_v150.safetensors%22&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=e01358d793ad6966166af8b3064953ad/20250907/us-east-1/s3/aws4_request&X-Amz-Date=20250907T013233Z&X-Amz-SignedHeaders=host&X-Amz-" -o ./illustriousXL_v01.safetensors

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6616M  100 6616M    0     0  49.0M      0  0:02:15  0:02:15 --:--:-- 53.2M


In [ ]:
#Cp dataset
!cp -r "/content/drive/MyDrive/cls_dataset.7z" /content/cls_dataset.7z
!cp -r "/content/drive/MyDrive/resized_img.7z" /content/resized_img.7z


In [ ]:
!sudo apt-get install p7zip-full

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!7z x /content/cls_dataset.7z -o/content/cls_dataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 167081667 bytes (160 MiB)

Extracting archive: /content/cls_dataset.7z
--
Path = /content/cls_dataset.7z
Type = 7z
Physical Size = 167081667
Headers Size = 1718
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     82% 94 - cls_dataset/cls_train_set_00089_.png                                              Everything is Ok

Folders: 1
Files: 113
Size:       167069628
Compressed: 167081667


In [ ]:
!7z x /content/resized_img.7z -o/content/resized_img


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 31615554 bytes (31 MiB)

Extracting archive: /content/resized_img.7z
--
Path = /content/resized_img.7z
Type = 7z
Physical Size = 31615554
Headers Size = 822
Method = LZMA2:24
Solid = +
Blocks = 1

  0%     23% 15 - resized_img/Still_010_001.txt                                        43% 22 - resized_img/Still_290_002.png                                        63% 26 - resized_img/Still_340_001.png                                        73

In [ ]:
#Install requirments
!pip install torch torchvision -q
!pip install transformers accelerate bitsandbytes datasets wandb -q
!pip install git+https://github.com/huggingface/diffusers.git -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 41.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 21.5 MB/s eta 0:00:00


In [ ]:
# covert ckpt to diffuser format
import torch
from diffusers import StableDiffusionXLPipeline
import os

civitai_model_path = "/content/illustriousXL_v01.safetensors"


output_dir = "/content/models"


def convert_model(model_path, output_path):
    """
    将单文件模型转换为 diffusers 格式的目录结构。
    """
    if not os.path.exists(model_path):
        print(f"错误：找不到模型文件 '{model_path}'。请检查路径是否正确。")
        return

    print(f"正在从单文件模型加载: {model_path}")
    print("这可能需要一些时间，请稍候...")

    try:

        pipeline = StableDiffusionXLPipeline.from_single_file(
            model_path,
            torch_dtype=torch.float16,
            use_safetensors=True
        )
        print("模型加载成功！")

    except Exception as e:
        print(f"加载模型时发生错误: {e}")
        print("请确保这是一个兼容的 Stable Diffusion (SD1.5 或 SDXL) 模型。")
        return

    print(f"正在将模型保存到目录: {output_path}")
    print("这将生成包括 unet, vae, text_encoder 在内的所有组件...")

    try:
        # safe_serialization=True 会确保所有组件都保存为 .safetensors 格式
        pipeline.save_pretrained(output_path, safe_serialization=True)
        print("-" * 50)
        print("转换成功！")
        print(f"您的训练模型已准备就绪，位于: {output_path}")
        print("现在您可以将训练脚本的模型路径指向这个新生成的文件夹。")
        print("-" * 50)

    except Exception as e:
        print(f"保存模型时发生错误: {e}")

if __name__ == "__main__":
    if "path/to/your" in civitai_model_path or "path/to/your" in output_dir:
        print("!!! 警告：请先修改脚本中的 'civitai_model_path' 和 'output_dir' 变量！")
    else:
        convert_model(civitai_model_path, output_dir)

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


正在从单文件模型加载: /content/illustriousXL_v01.safetensors
这可能需要一些时间，请稍候...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

merges.txt: 0.00B [00:00, ?B/s]

...

config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

模型加载成功！
正在将模型保存到目录: /content/models
这将生成包括 unet, vae, text_encoder 在内的所有组件...
--------------------------------------------------
转换成功！
您的训练模型已准备就绪，位于: /content/models
现在您可以将训练脚本的模型路径指向这个新生成的文件夹。
--------------------------------------------------


In [ ]:
# !git clone https://github.com/huggingface/diffusers.git
#you might want do upgrade diffuser to dev ver.
# !pip install git+https://github.com/huggingface/diffusers.git -q

In [ ]:
#If you want to use wandb to lod
os.environ["WANDB_API_KEY"] = ""

In [ ]:

import os
import subprocess
!accelerate config default

print("\n✅ Accelerate 配置完成！准备启动训练...\n")
#A100 bs改8


accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml

✅ Accelerate 配置完成！准备启动训练...



In [ ]:
#cp script
!cp "/content/drive/MyDrive/train_dreambooth_lora_sdxl.py" "/content/train_dreambooth_lora_sdxl.py"

In [ ]:
#Start to train
!accelerate launch train_dreambooth_lora_sdxl.py \
  --pretrained_model_name_or_path="/content/models" \
  --instance_data_dir="/content/resized_img/resized_img" \
  --class_data_dir="/content/cls_dataset/cls_dataset" \
  --output_dir="/content/drive/MyDrive/4_chars/output" \ 
  --with_prior_preservation --prior_loss_weight=0.48 \
  --mixed_precision="bf16" \
  --instance_prompt="masterpiece,best quality, Jogasaki_Noa" \
  --class_prompt="masterpiece,best quality,1 girl," \
  --resolution=1024 \
  --random_flip \
  --snr_gamma=5 \
  --train_batch_size=2 \
  --gradient_checkpointing \
  --gradient_accumulation_steps=4 \
  --learning_rate=1e-4 \
  --rank=128 \
  --lr_scheduler="cosine" \
  --enable_xformers_memory_efficient_attention \
  --report_to="wandb" \
  --lr_warmup_steps=0 \
  --max_train_steps=3400 \
  --validation_prompt="masterpiece,best quality, Kurobe_Nanoka" \
  --num_validation_images=3 \
  --validation_epochs=34 \
  --seed=42 \
  --use_8bit_adam \
  --checkpoints_total_limit=5 \
  --checkpointing_steps=200 \
  --local_config_file_name="file_name" \
  --local_config_text_name="text" \
  --bucket_size="1280x640" \

  # --train_text_encoder \
  # --text_encoder_lr=3.5e-5 \


2025-09-07 15:24:44.353324: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757258684.375595  171309 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757258684.382381  171309 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757258684.399412  171309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757258684.399440  171309 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757258684.399443  171309 computation_placer.cc:177] computation placer alr